In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost
import shap

In [2]:
pd.options.display.max_columns = 30

In [3]:
df = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv' , delimiter='\t')
df

In [4]:
df.info()

In [5]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

In [6]:
df.describe()

In [7]:
df.drop(['Z_CostContact', 'Z_Revenue'], inplace=True, axis=1) #They contain single value only(STD = 0)

In [8]:
df.isna().sum()

In [9]:
df = df[df['Income'].notnull()]

In [10]:
cat_feat = [feature for feature in df.columns if (df[feature].dtype == 'O' or df[feature].dtype == bool)]
cat_feat

In [11]:
from sklearn.cluster import KMeans

In [16]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df[['Income']])
    wcss.append(kmeans.inertia_)

In [17]:
plt.plot(range(1,11) , wcss)

In [18]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)

In [19]:
df.columns

In [ ]:
X = df.loc[: , ['ID', 'Year_Birth', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Response']].values
y = df.loc[: ,'Income'].values
model = xgboost.XGBClassifier().fit(X, y)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])